In [10]:
DOWNLOAD_FOLDER = "./data/tabroom"
!mkdir -p {DOWNLOAD_FOLDER}/error
first = 27000
last = 28000

In [1]:
from tqdm.notebook import tqdm
import scipy as sc

In [2]:
import urllib.request
import shutil
def downloadTourn(id):
  with urllib.request.urlopen(f"https://tabroom.com/api/download_data?tourn_id={id}?") as response, open(f"{DOWNLOAD_FOLDER}/{id}.json", 'wb') as f:
    shutil.copyfileobj(response, f)

In [3]:
import os.path
def notDownloaded(id):
  return not os.path.isfile(f"{DOWNLOAD_FOLDER}/{id}.json")

In [6]:
# last = 25874

downloadList = [id for id in range(last, first, -1)] # if notDownloaded(id)]

In [7]:

from urllib.error import URLError

for i in tqdm(downloadList):
  try:
    downloadTourn(i)
  except TimeoutError:
    print(f"Request for {i} timed out")
  except URLError:
    print(f"Request for {i} timed out")
  # time.sleep(1)

  0%|          | 0/500 [00:00<?, ?it/s]

In [12]:
import json

for i in tqdm(range(first, last + 1)):
  path = f'{DOWNLOAD_FOLDER}/{i}.json'
  try:
    with open(path) as f:
      try:
        tourn = json.load(f)
        if 'error' in tourn: raise ValueError
      except Exception:
        shutil.move(path, f'{DOWNLOAD_FOLDER}/error/{i}.json')
  except FileNotFoundError:
    pass

  0%|          | 0/1001 [00:00<?, ?it/s]

In [ ]:
import glob
import re
tournIds = sorted([int(re.search(r'/(\d+)\.json', path).group(1)) for path in glob.glob(f'{DOWNLOAD_FOLDER}/*.json')])

In [41]:
def removeLocales(obj: dict):
  if type(obj) is list:
    for item in obj: removeLocales(item)
  elif type(obj) is dict:
    if 'locale' in obj: del obj['locale']
    for value in obj.values(): removeLocales(value)

with open('./data/tabroom/22381.json') as f:
  data = json.load(f)
  print(len(json.dumps(data)))
  removeLocales(data)
  print(len(json.dumps(data)))
  with open('./data/example.json', 'w+') as f2:
    json.dump(data, f2, indent=2)

7636410
6463770


In [37]:
for key in data:
  print(type(data[key]))

<class 'str'>
<class 'NoneType'>
<class 'list'>
<class 'str'>
<class 'NoneType'>
<class 'str'>
<class 'str'>
<class 'list'>
<class 'NoneType'>
<class 'str'>
<class 'NoneType'>
<class 'int'>
<class 'list'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'NoneType'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


False

In [ ]:
import json
with open("toc.json") as f:
  data = json.load(f)
data

In [ ]:
import json
with open("tocF.json") as f:
  dataF = json.load(f)
dataF

In [5]:
policy = data['categories'][3]['events'][0]

In [40]:
from collections import Counter

entries = {}
judges = {}
debates = []

for round in policy['rounds']:
  for section in round['sections']:
    debate = {}
    ballots = section['ballots']
    if len(ballots) != 2: continue # TODO implement processing for rounds with something other than 2 ballots
    for ballot in ballots:
      if 'judge' in ballot and ballot['judge'] not in judges:
        judges[ballot['judge']] = f"{ballot['judge_first']} {ballot['judge_last']}"
      if ballot['entry'] not in entries:
        entries[ballot['entry']] = {
          'entry_name': ballot['entry_name'], 
          'entry_code': ballot['entry_code'], 
          'people': set() 
        }
      match ballot['side']:
        case 1: side = "aff"
        case 2: side = "neg"
        case _: side = None
      debate[side] = { 'team': entries[ballot['entry']], 'points': {} }

      if 'scores' in ballot:
        for score in ballot['scores']:
          match score:
            case {'tag': 'point'}:
              entries[ballot['entry']]['people'].add(score['speaker'])
              debate[side]['points'][score['speaker']] = score['value']
            case {'tag': 'winloss', 'value': 1}:
              debate['winner'] = side
    debates.append(debate)
    

In [44]:
def set_default(obj):
    if isinstance(obj, set):
        return list(obj)
    raise TypeError

with open('./data/tocDebates.json', 'w') as f:
  f.write(json.dumps(debates, indent=2, default=set_default))

In [ ]:
winCounts = Counter()
for debate in debates:
  if 'winner' not in debate: continue
  winCounts[debate[debate['winner']]['team']['entry_code']] += 1
winCounts

In [26]:
winCounts = Counter(map(lambda d: d['winner'] if 'winner' in d else "NONE", debates))
total = winCounts['NEG'] + winCounts['AFF']
sc.stats.binomtest(360, total, p=0.5)
# sc.stats.binomtest(total // 2, total, p=winCounts['NEG']/total))

BinomTestResult(k=360, n=700, alternative='two-sided', proportion_estimate=0.5142857142857142, pvalue=0.47270122899464756)

In [42]:
import base64
import zlib
from IPython.display import HTML

def show(e: str):
  # return base64.decode(e).decode('zlib')
  return HTML(zlib.decompress(base64.b64decode(e)).decode('utf-8'))

In [43]:
policy = dataF['categories'][3]['events'][0]

In [44]:
show(policy['rounds'][11]['sections'][0]['ballots'][0]['scores'][0]['content'])